In [27]:
import ccxt
from pytrends.request import TrendReq
import numpy as np
import pandas as pd
from datetime import datetime
import ta as ta
from kafka import KafkaProducer
producer = KafkaProducer(bootstrap_servers='10.204.131.11:9092')

ModuleNotFoundError: No module named 'kafka'

In [ ]:
# Faz sentido ter isto como um script separado? Devia ser chamado no momento em que a informação é enviada para o hdfs ? E usar um fake producer?


In [42]:
#leitura de data do hdsf
#copia dados para dados treino
data = pd.read_csv("data.csv", index_col="date_time")
data.to_csv('data_treino.csv', index=True) 
data

FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

In [38]:
# Specify the symbol for Bitcoin
symbol = 'BTC/USDT'
exchange = ccxt.binance()
# Fetch the ticker data
ticker = exchange.fetch_ticker(symbol)


# Fetch the OHLCV (Open, High, Low, Close, Volume) data
ohlcv = exchange.fetch_ohlcv(symbol, timeframe='4h', limit=1)  # daily data, most recent

# Extract the required information
current_price = ticker['last']
volume = ticker['quoteVolume']  # 24h volume in quote currency (USDT)
open_price = ohlcv[0][1]
high_price = ohlcv[0][2]
close_price = ohlcv[0][4]
low_price= ohlcv[0][3]

print(f"Current Price: {current_price}")
print(f"Volume: {volume}")
print(f"Open Price: {open_price}")
print(f"High Price: {high_price}")
print(f"Low Price: {low_price}")
print(f"Close Price: {close_price}")


Current Price: 67759.01
Volume: 568523929.0791715
Open Price: 67766.84
High Price: 67910.55
Low Price: 67744.0
Close Price: 67759.0


In [39]:
# Initialize pytrends
pytrends = TrendReq(hl='en-US', tz=360)
pd.set_option('future.no_silent_downcasting', True)
# Specify the keyword
keyword = "Bitcoin"

# Vai buscar as trends da ultima semana para nao alterar o impacto
pytrends.build_payload([keyword], cat=0, timeframe='now 7-d', geo='', gprop='')

,Bitcoin,isPartial
date,,
2024-05-26 01:00:00,72,False
2024-05-26 02:00:00,68,False
2024-05-26 03:00:00,65,False
2024-05-26 04:00:00,67,False
2024-05-26 05:00:00,68,False
...,...,...
2024-06-01 21:00:00,70,False
2024-06-01 22:00:00,72,False
2024-06-01 23:00:00,64,False


In [ ]:
# Fetch the interest over time data
g_trends = pytrends.interest_over_time()
g_trends_value= g_trends.iloc[-1]['Bitcoin']
g_trends_value

In [41]:

#adicionar novos dados ao nosso amigo data e compilar valores
df = pd.DataFrame({
    'date_time': datetime.now().strftime('%Y/%m/%d %H:%M'),
    'open': open_price,
    'high': high_price,
    'low': low_price,
    'close': close_price,
    'volume': volume,
    'bitcoin': g_trends_value
}, index=[0])
df.set_index('date_time',inplace=True)
data.tail()

#data.to_csv('data.csv', index=True)

,Bitcoin,isPartial,date_time,open,high,low,close,volume
166,63.0,False,NaN,NaN,NaN,NaN,NaN,NaN
167,64.0,False,NaN,NaN,NaN,NaN,NaN,NaN
168,55.0,True,NaN,NaN,NaN,NaN,NaN,NaN
169,NaN,NaN,2024-06-02 02:49:56,67766.84,67910.55,67744.0,67806.21,5.805123e+08
170,NaN,NaN,2024-06-02 02:49:56,67766.84,67910.55,67744.0,67806.21,5.805123e+08


In [ ]:
df

In [36]:
data = pd.concat([data, df], ignore_index=False)
data.tail()

,Bitcoin,isPartial,date_time,open,high,low,close,volume
166,63.0,False,NaN,NaN,NaN,NaN,NaN,NaN
167,64.0,False,NaN,NaN,NaN,NaN,NaN,NaN
168,55.0,True,NaN,NaN,NaN,NaN,NaN,NaN
169,NaN,NaN,2024-06-02 02:49:56,67766.84,67910.55,67744.0,67806.21,5.805123e+08
170,NaN,NaN,2024-06-02 02:49:56,67766.84,67910.55,67744.0,67806.21,5.805123e+08


In [ ]:
#Criação de labels de subida ou descida de preço
data['label'] = np.where(data['close'].ge(data['close'].shift()),1,0)
#features analise tecnica
data['rsi_indicator'] = ta.momentum.rsi(data.close)
data['wma_indicator']= ta.trend.wma_indicator(data.close)
data['ema_indicator']= ta.trend.ema_indicator(data.close)
precos_reais= data.dropna()
data.tail(10)

In [ ]:
data.to_csv('data.csv', index=True) 

In [ ]:
producer.flush()
producer.close()